# Example Notebook to run pydicer

This notebooks provides a basic example to run the pydicer pipeline using some test data.

In [1]:
import sys
sys.path.insert(0, "..")

from pathlib import Path

from pydicer.input.test import TestInput
from pydicer.preprocess.data import PreprocessData
from pydicer.convert.data import ConvertData
from pydicer.visualise.data import VisualiseData
from pydicer.dataset.preparation import PrepareDataset
from pydicer.analyse.data import AnalyseData

## Setup directories

First we'll set up some directories in which to fetch and convert our data. Change the `directory`
location to a folder on your system where you'd like to work with this data.

In [2]:

directory = Path("./data")
directory.mkdir(exist_ok=True, parents=True)

dicom_directory = directory.joinpath("dicom")
dicom_directory.mkdir(exist_ok=True, parents=True)

nifti_directory = directory.joinpath("nifti")
nifti_directory.mkdir(exist_ok=True, parents=True)

clean_directory  = directory.joinpath("clean")


## Fetch some data

A TestInput class is provided in pydicer to download some sample data to work with. Several other
input classes exist if you'd like to retrieve DICOM data for conversion from somewhere else, [see 
the docs for information on how these work](https://australiancancerdatanetwork.github.io/pydicer/html/input.html).

In [ ]:
test_input = TestInput(dicom_directory)
test_input.fetch_data()

## Preprocess the data

Before pydicer goes ahead and converts all the data, it first runs through it all once to figure
out how it is linked and move invalid data to the quarantine folder.

In [10]:
preprocessed_data = PreprocessData(dicom_directory, nifti_directory)
preprocessed_result = preprocessed_data.preprocess()

# Convert the data

Next we convert all the DICOM data into Nifti format. Check out the nifti folder to see the files
arrive as they are converted!

Alongside the Nifti files there are also a few other files made available. The JSON files which is
output stores all of the meta data from the original DICOM so that you can use it later.

In [11]:
convert_data = ConvertData(preprocessed_result, output_directory=nifti_directory)
convert_data.convert()

2022-02-10 15:28:28.522 | DEBUG    | platipy.dicom.io.rtstruct_to_nifti:transform_point_set_from_dicom_struct:134 - Converting structure 0 with name: Marked_Iso
2022-02-10 15:28:28.523 | DEBUG    | platipy.dicom.io.rtstruct_to_nifti:transform_point_set_from_dicom_struct:152 - This is not a closed planar structure, skipping.
2022-02-10 15:28:28.524 | DEBUG    | platipy.dicom.io.rtstruct_to_nifti:transform_point_set_from_dicom_struct:134 - Converting structure 1 with name: Calc_pt
2022-02-10 15:28:28.526 | DEBUG    | platipy.dicom.io.rtstruct_to_nifti:transform_point_set_from_dicom_struct:152 - This is not a closed planar structure, skipping.
2022-02-10 15:28:28.545 | DEBUG    | platipy.dicom.io.rtstruct_to_nifti:transform_point_set_from_dicom_struct:134 - Converting structure 2 with name: External
2022-02-10 15:28:31.390 | DEBUG    | platipy.dicom.io.rtstruct_to_nifti:transform_point_set_from_dicom_struct:134 - Converting structure 3 with name: GTV1
2022-02-10 15:28:32.169 | DEBUG    | 

# Visualise the data

Nifti format is great, but it can be a bit time consuming to load each file in to 3D Slicer or a
similar tool to look at it. So in this step some visualisations providing snapshots of the images,
structures and dose will be saved along side the converted files in PNG format.

In [12]:
visualise_data = VisualiseData(nifti_directory)
visualise_data.visualise()

# Prepare a dataset

Datasets which are extracted in DICOM format can often be a bit messy and require some cleaning up
after conversion. Exactly what data objects to extract for the clean dataset will differ by project
but here we use a somewhat common approach of extracting the latest Structure Set for a patient and
the image linked to that.


In [13]:
prepare_dataset = PrepareDataset(directory)
prepare_dataset.prepare("clean", "rt_latest_struct")

# Analyse the dataset

We can compute some radiomics values on the converted dataset and extract a DataFrame containing those.

In [3]:
analyse = AnalyseData(directory, "clean")
analyse.compute_radiomics()
df = analyse.get_all_computed_radiomics_for_dataset()